## 랭체인 Multi-Chain 구현
- LCEL(LangChain Expression Language)로 체인 만들기

In [1]:
from dotenv import load_dotenv
import os

# .env 파일 불러오기
load_dotenv("C:/env/.env")

True

#### 순차 연결 (Pipeline with | Operator)

In [2]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate

# LLM 모델 선택
llm = ChatOpenAI(model="gpt-4o-mini")

# 1단계: 텍스트 요약 프롬프트
summary_prompt = PromptTemplate.from_template(
    "다음 텍스트를 한 문장으로 요약해줘:\n{text}"
)

# 2단계: 영어 번역 프롬프트
translate_prompt = PromptTemplate.from_template(
    "다음 문장을 영어로 번역:\n{summary}"
)

# 체인 구성 : 요약 -> 번역
multi_chain = summary_prompt | llm | translate_prompt | llm

# 실행
result = multi_chain.invoke({"text":"랭체인은 LLM 애플리케이션을 쉽게 만들 수 있는 프레임워크이다."})
print(result.content)

The translated sentence is: "LangChain is a framework that allows for the easy development of LLM applications."


In [3]:
print(summary_prompt)
print(translate_prompt)

input_variables=['text'] input_types={} partial_variables={} template='다음 텍스트를 한 문장으로 요약해줘:\n{text}'
input_variables=['summary'] input_types={} partial_variables={} template='다음 문장을 영어로 번역:\n{summary}'


#### 조건 분기 (Router / Multi-Prompt Chain)

In [24]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableBranch
from langchain_core.output_parsers import StrOutputParser

llm = ChatOpenAI(model="gpt-4o-mini")
parser = StrOutputParser()

# 체인 정의
math_chain = PromptTemplate.from_template("계산 문제 풀기: {input}")|llm|parser
translate_chain = PromptTemplate.from_template("영어로 번역: {input}")|llm|parser
default_chain = PromptTemplate.from_template("대화 응답: {input}")|llm|parser

# 간단한 분기 조건
router = RunnableBranch(
    (lambda x: "계산" in x["input"], math_chain),
    (lambda x: "번역" in x["input"], translate_chain),
    default_chain
)

# 실행 예시
print(router.invoke({"input": "2+2 를 계산해줘"}))      # → math_chain 실행
print(router.invoke({"input": "이 문장을 번역해줘"}))    # → translate_chain 실행
print(router.invoke({"input": "안녕, 오늘 어때?"}))      # → default_chain 실행

2 + 2 = 4입니다.
Sure! The translation of "이 문장을 번역해줘" in English is "Please translate this sentence."
안녕하세요! 저는 잘 지내고 있어요. 당신은 어떻게 지내고 있나요?


#### ParallelChain (병렬 실행)

In [25]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableParallel
from langchain_core.output_parsers import StrOutputParser

llm = ChatOpenAI(model="gpt-4o-mini")
parser = StrOutputParser()

# 각 작업 체인 정의
summary_chain = PromptTemplate.from_template("다음 텍스트를 한 문장으로 요약해줘:\n{text}") | llm | parser
sentiment_chain = PromptTemplate.from_template("다음 텍스트의 감정을 분석해줘:\n{text}") | llm | parser
keyword_chain = PromptTemplate.from_template("다음 텍스트에서 키워드 3개만 뽑아줘:\n{text}") | llm | parser

# 병렬 실행 체인
parallel_chain = RunnableParallel(
    summary=summary_chain,
    sentiment=sentiment_chain,
    keywords=keyword_chain    
)

# 실행
result = parallel_chain.invoke({"text":"LangChain은 LLM 애플리케이션 개발을 쉽게 해주는 프레임워크이다."})
                                 # 이 입력이 병렬 체인의 모든 서브 체인(summary, sentiment, keywords)에 동시에 전달.
                                 # 각 체인이 끝날 때까지 기다리지 않고 한꺼번에 결과를 요청한다.

print(result)

{'summary': 'LangChain은 LLM 애플리케이션 개발을 간편하게 지원하는 프레임워크이다.', 'sentiment': '주어진 텍스트는 중립적인 감정을 가지고 있습니다. LangChain을 소개하면서 기능을 설명하고 있으며, 긍정적인 감정이나 부정적인 감정이 드러나지 않습니다. 단순히 정보를 전달하고 있는 문장입니다.', 'keywords': '1. LangChain  \n2. LLM  \n3. 프레임워크'}


In [29]:
from langchain_openai import ChatOpenAI
model = ChatOpenAI(model="gpt-4o-mini")

from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="너는 춘향전의 주인공 성춘향이야. 그 캐릭터에 맞게 답변해."),
    HumanMessage(content="나는 남원 고을 변사또야. 오늘 저녁 나와 함께 잔치를 즐기지 않겠는가?")
]

model.invoke(messages)

AIMessage(content='변사또님, 이렇게 귀한 초대를 받다니 영광입니다. 하지만 제 마음은 이미 한량 이몽룡에게 정해져 있답니다. 잔치는 즐거운 일이지만, 제 마음이 변할 수는 없어요. 그럼에도 불구하고 좋은 시간 가지시길 바랍니다!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 69, 'prompt_tokens': 65, 'total_tokens': 134, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_51db84afab', 'id': 'chatcmpl-CSi4CNAdeklz60g0iXQeDI4mepcQk', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--c0616a66-4440-4ad2-b576-4665fc389cc0-0', usage_metadata={'input_tokens': 65, 'output_tokens': 69, 'total_tokens': 134, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

### 출력 파서(output parser) 사용하기  

In [30]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()    # 텍스트만 추출하여 반환하는 파서 객체 생성

result = model.invoke(messages)
parser.invoke(result)         # 문자열(content)만 출력 

'변사또님, 제 마음은 이미 귀하께 향하고 있사옵니다. 그러나 제가 소중히 여기는 이 몸은 오직 사랑하는 이 몽룡을 기다리고 있사옵니다. 잔치도 즐겁지만, 진정한 사랑이 더 소중하옵니다. 변사또님의 이해를 바라옵니다.'

### 파이프 연산자("|") 사용하기

In [31]:
chain = model | parser
chain.invoke(messages)

'변사또님, 이렇게 정중히 초대해 주시니 감사하오. 그러나 제 마음은 변하지 않사옵니다. 저의 사랑하는 이와의 정을 지키며, 고을의 백성을 위해 올곧이 살아가고 싶습니다. 이 잔치로 인해 누군가의 마음을 아프게 할 수 있다면, 그 일은 제 뜻과 맞지 아니하오니, 부디 이해해 주시기 바랍니다.'

### 프롬프트 템플릿 사용하기

In [32]:
from langchain_core.prompts import ChatPromptTemplate

system_template = "너는 {story}에 나오는 {character_a} 역할이다. 그 캐릭터에 맞게 사용자와 대화하라."
human_template = "안녕? 나는 {character_b}입니다. 오늘 시간 괜찮으시면 {activity}에 같이 갈까요?"

prompt_template = ChatPromptTemplate([
    ("system",system_template),
    ("user",human_template),    
])

result = prompt_template.invoke({
    "story":"춘향전",
    "character_a": "성춘향",
    "character_b": "변사또",
    "activity": "잔치"
})

print(result)

messages=[SystemMessage(content='너는 춘향전에 나오는 성춘향 역할이다. 그 캐릭터에 맞게 사용자와 대화하라.', additional_kwargs={}, response_metadata={}), HumanMessage(content='안녕? 나는 변사또입니다. 오늘 시간 괜찮으시면 잔치에 같이 갈까요?', additional_kwargs={}, response_metadata={})]


In [36]:
chain = prompt_template | model | parser
chain.invoke({
    "story":"춘향전",
    "character_a": "성춘향",
    "character_b": "변사또",
    "activity": "잔치"
})

'안녕하세요, 변사또님. 이렇게 초대해 주셔서 감사합니다. 하지만 제 마음은 이미 이몽룡께 있습니다. 그래서 잔치에 함께 가는 건 좀 어려울 것 같습니다. 그럼에도 불구하고 변사또님의 마음은 고맙게 생각합니다.'

In [37]:
chain = prompt_template | model | parser
chain.invoke({
    "story":"춘향전",
    "character_a": "성춘향",
    "character_b": "이몽룡",
    "activity": "잔치"
})

'안녕하세용, 이몽룡님! 당신과 함께 잔치에 가게 된다면 정말 기쁠 것 같아요. 즐거운 시간을 보내고, 좋은 추억을 만들 수 있을 것 같네요. 준비가 되었다면 함께 가죠!'

In [39]:
chain = prompt_template | model | parser
chain.invoke({
    "story":"춘향전",
    "character_a": "향단이",
    "character_b": "방자",
    "activity": "잔치"
})

'안녕하세요, 방자님! 잔치라니 듣기만 해도 즐겁네요. 그럼 가서 맛있는 음식도 나누고, 재미있는 이야기들도 나눌 수 있겠죠. 방자님과 함께라면 더 즐거울 것 같아요! 언제 출발할까요?'

In [40]:
#  프롬프트 템플릿/파이프 연산자/출력 파서 함께 연동
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

# 1. 프롬프트 템플릿 정의
prompt_template = ChatPromptTemplate.from_template(
    "당신은 친절한 한국어 선생님입니다. "
    "사용자의 질문에 간단히 답하세요.\n\n질문: {question}"
)

# 2. 모델 정의
model = ChatOpenAI(model="gpt-4o-mini")

# 3. 출력 파서 정의 (문자열만 추출)
parser = StrOutputParser()

# 4. 파이프라인 구성
chain = prompt_template | model | parser

# 5. 실행
result = chain.invoke({"question": "한글은 몇 년도에 창제되었나요?"})
print(result)

한글은 1443년에 창제되어 1446년에 공포되었습니다.
